In [ ]:
# !D:\olga_bassant_roadmap\B2B_project\B2B\myenv\Scripts\activate && pip install sentence-transformers
!D:\olga_bassant_roadmap\B2B_project\B2B\myenv\Scripts\activate


In [ ]:
import sys
print(sys.executable)


In [ ]:
# !{sys.executable} -m pip install sentence-transformers


In [ ]:
# ! pip install sentence-transformers


In [ ]:
import sentence_transformers
print(sentence_transformers.__version__)


In [ ]:
# import spacy
from langchain_community.llms import Ollama

In [ ]:
llm4generator_extractor= Ollama(model="llama2", temperature=0.7)
# llm4generator_extractor= Ollama(model="deepseek-r1", temperature=0.7)
llm4eval= Ollama(model="deepseek-r1", temperature=0.7)


In [ ]:
# Read job description from text file
with open("Job Description1.txt", "r") as file:
    job_description = file.read()

with open("cv.txt", "r") as file:
    cv_text = file.read()

In [ ]:
prompt4extract_data = f"""

You are an intelligent assistant designed to extract structured data from a CV. Your task is to **analyze the entire CV** and extract **all relevant information**, ensuring that no key details are missed. If any information is unavailable, mark it as "Not Available." The extracted information should be formatted **clearly and concisely in plain text.**  

---

### **Data to Extract:**

1. **Name**  
   - Extract the **full name** of the individual.

2. **Contact Information**  
   - Email address  
   - Phone number  

3. **Education History**  
   - Degree(s) earned (e.g., Bachelor of Science in Computer Science)  
   - Institution(s) attended (e.g., University of XYZ)  
   - Year(s) of graduation, if available  

4. **Work Experience**  
   Extract **all job experiences**, even if they are embedded within descriptions, projects, or skills sections. Ensure that each entry includes:  
   - **Job Title** (e.g., Software Engineer)  
   - **Company Name** (e.g., ABC Corp)  
   - **Duration of Employment** (e.g., Jan 2020 – Dec 2022), if available  
   - **Job Responsibilities & Achievements** (if available)  

   🔹 **Include all experiences, even if they appear in different formats** (e.g., paragraphs, bullet points, or tables).  
   🔹 **If a duration is missing, extract at least the job title and company.**  

5. **Skills**  
   Extract **all mentioned skills**, including those found in job descriptions, project details, or other sections. Categorize them as follows:  
   - **Programming Languages** (e.g., Python, C++, Java)  
   - **Frameworks & Libraries** (e.g., TensorFlow, Flask, React)  
   - **Tools & Technologies** (e.g., Docker, AWS, Git)  
   - **Soft Skills** (e.g., Leadership, Communication, Problem-Solving)  

---

### **Output Format:**  
Name: [Full Name]

Contact Information:
Email: [Email Address]
Phone: [Phone Number]

Education History:

[Degree], [Institution], [Year of Graduation]
[Degree], [Institution], [Year of Graduation]
Work Experience:

[Job Title], [Company], [Duration]
Responsibilities:

[Responsibility 1]
[Responsibility 2]
[Job Title], [Company], [Duration]
Responsibilities:

[Responsibility 1]
[Responsibility 2]
Skills:
Programming Languages:

[Skill 1]
[Skill 2]
Frameworks & Libraries:

[Skill 1]
[Skill 2]
Tools & Technologies:

[Skill 1]
[Skill 2]
Soft Skills:

[Skill 1]
[Skill 2]


---

### **Additional Instructions:**

1. **Ensure Thorough Analysis**  
   - Examine **all sections**, including headers, bullet points, and paragraphs.  
   - Extract work experience **even if not explicitly labeled** as "Work Experience."  

2. **Handle Multiple Entries**  
   - Include **all** education and work experience entries.  

3. **Extract Implicit Skills**  
   - Identify **skills hidden in job descriptions** and responsibilities.  

4. **Mark Missing Information**  
   - If any detail (e.g., graduation year) is unavailable, output **"Not Available."**  

5. **Ensure Accuracy & Formatting**  
   - Structure the output **cleanly and consistently.**  

---

### **Example CV Text**  

Name: John Smith

Contact Information:
Email: john.smith@example.com
Phone: +1 555 123 4567

Education History:

Master of Science in Computer Science, Stanford University, 2021
Bachelor of Science in Electrical Engineering, MIT, 2019
Work Experience:

Senior Software Engineer, Tech Innovators Inc., Jan 2022 – Present
Responsibilities:

Led backend development for AI-driven products
Improved system scalability by 40% using cloud technologies
Software Engineer, CodeCraft LLC, Jun 2019 – Dec 2021
Responsibilities:

Developed web applications using React and Flask
Optimized database queries to enhance performance
Skills:
Programming Languages:

Python
Java
JavaScript
Frameworks & Libraries:

Flask
React
TensorFlow
Tools & Technologies:

Docker
AWS
Git
Soft Skills:

Leadership
Team Collaboration
Problem-Solving


---

### **Task:**  
Now, extract the structured information from the following CV:



"""

In [ ]:
# function used to run llm prompt to extract the data of the  cv file
def extract_cv_info_with_llm(cv_text,prompt4extract_data):
    # Define a prompt for the LLM to extract structured information
    prompt4extract_data=prompt4extract_data+cv_text
    # extract cv data
    cv_data = llm4generator_extractor(prompt4extract_data)
    return cv_data
    
with open("cv.txt", "r") as file:
    cv_text = file.read()

cv_data = extract_cv_info_with_llm(cv_text,prompt4extract_data)
print("Extracted CV Data:", cv_data)

In [ ]:
  # Prepare the prompt
phase1_prompt4genrator = f"""
I need a cover letter based on their CV and the job description provided. The letter should emphasize the candidate's relevant skills, 
experience, and achievements while tailoring the content to the requirements of the job. Below are the details:
"""

In [ ]:
# Function to generate a cover letter
def generate_cover_letter(cv_data, job_description):
    # concate all the info for the project
    prompt = phase1_prompt4genrator+f"""
   
    CV:
    {cv_data}
    
    Job Description:
    {job_description}

    you must ensure that it concludes with the CV owner's name as extracted from the CV. The name should appear at the very end of the letter.


    """
    # Generate the cover letter using LLaMA 2
    cover_letter = llm4generator_extractor(prompt)
    return cover_letter
# generate the cover letter
cover_letter = generate_cover_letter(cv_data, job_description)
print("Generated cover Letter:\n", cover_letter)

In [ ]:
cover_letter
with open("gnerate_cover_1.txt", "w") as file:
    file.write(cover_letter)

In [ ]:
# Function to evaluate the cover letter
def evaluate_cover_letter(cv_data, job_description, cover_letter):
    # Prepare the evaluation prompt
    evaluation_prompt = f"""
    Please evaluate the following cover letter based on the criteria listed below. For each point, provide a score from 1 (poor) to 5 (excellent), along with a brief explanation or examples supporting your score. Then, provide detailed feedback on the letter, including specific suggestions for improvement where necessary. Finally, include a summary stating whether the letter is **"Approved"** or **"Needs Improvement."**


    **Criteria for Evaluation:**  
    1. **Tailored to the Role/Context:**  
    - Does the letter address the specific job, role, or context clearly?  
    - Is it customized to the recipient (e.g., mentioning the company, university, or field of interest)?  

    2. **Strong Opening:**  
    - Does the letter start with an engaging introduction?  
    - Is the relationship between the writer and the subject of the letter clearly established?  

    3. **Showcases Value:**  
    - Does the letter effectively highlight the subject's key strengths, skills, or achievements?  
    - Are these strengths relevant to the context or role in question?  

    4. **Concrete Examples and Achievements:**  
    - Are specific examples or achievements mentioned to back up claims about the subject?  
    - Are these examples impactful and quantifiable (e.g., using metrics, measurable results)?  

    5. **Well-Structured and Concise:**  
    - Is the letter logically organized and easy to follow?  
    - Is it concise, avoiding unnecessary information while covering all critical points?  

    6. **Professional Tone and Authenticity:**  
    - Does the tone remain professional throughout the letter?  
    - Does the letter feel genuine and sincere in its praise?  

    7. **Strong Closing:**  
    - Does the letter end with a compelling summary and cover?  
    - Is there a clear call to action or offer for further communication?  

    8. **Error-Free and Polished:**  
    - Is the letter free from grammatical, spelling, or formatting errors?  
    - Does the formatting look clean and professional?  

    ---

    **Output Format:**  
    1. **Score for Each Criterion (1–5)**: Provide a score and brief explanation for each criterion.  
    2. **Detailed Feedback**:  
    - Highlight specific strengths of the letter.  
    - Identify areas needing improvement, with actionable suggestions for refinement.  
    3. **Summary**:  
    - Indicate whether the letter is **"Approved"** or **"Needs Improvement."**  
    - If **"Needs Improvement,"** explain what must be addressed to meet approval. 
 

   
    ---
Here are the needed data

        CV:
        {cv_data}
        
        Job Description:
        {job_description}
        
        cover Letter:
        {cover_letter}


        """

    # use the deepseek  model to evaluate the letter
    feedback = llm4eval(evaluation_prompt)
    return feedback


In [ ]:
def refined_cover_letter(feedback,cv_data,job_description,current_letter):
        refinement_prompt = f"""
        The evaluator provided the following feedback on your previous cover letter:
        {feedback}

        Based on this feedback, refine the cover letter to improve its quality while addressing the issues highlighted.
        Ensure it remains professional and well-aligned with the job description.

        Here are the details for context:

        **Candidate's CV**:
        {cv_data}

        **Job Description**:
        {job_description}

        **Current Cover Letter**:
        {current_letter}

        Generate a new and improved version of the cover letter.
        
        you must only add the new cover letter dont add the feedback or job Description


        """
        
        # Generate the refined cover letter
        current_letter = llm4generator_extractor(refinement_prompt)
        return current_letter

In [ ]:
def feedback_loop(cv_data, job_description, max_iterations=2):
    current_letter = generate_cover_letter(cv_data, job_description)

    for iteration in range(max_iterations):
        print(f"Iteration {iteration + 1}: Evaluating the letter...\n")
        # get the feedback 
        feedback = evaluate_cover_letter(cv_data, job_description, current_letter)
        
        # Display feedback
        # print(f"Feedback from evaluator:\n{feedback}\n")
        
        # Check if the letter is approved
        if "Approved" in feedback:
            print("The letter has been approved")
            return current_letter
    
        else:
            print("Refining the cover letter based on feedback...\n") 
            print("****************************************************************")
            current_letter=refined_cover_letter(feedback,cv_data,job_description,current_letter)

    print("Max iterations reached. Returning the latest version of the letter.")
    return current_letter


In [ ]:
cover_letter=feedback_loop(cv_data, job_description, max_iterations=2)
with open("cover_letter4_atterations.txt", "w") as file:
    file.write(cover_letter)

In [ ]:
# # Feedback loop to refine the cover letter
# def feedback_loop(cv_data, job_description, max_iterations=2):
#     current_letter = generate_cover_letter(cv_data, job_description)
    
#     for iteration in range(max_iterations):
#         print(f"Iteration {iteration + 1}: Evaluating the letter...\n")
        
#         # Get feedback from the evaluator
#         feedback = evaluate_cover_letter(cv_data, job_description, current_letter)
        
#         # Display feedback
#         print(f"Feedback from evaluator:\n{feedback}\n")
        
#         # Check if the letter is approved
#         if "Approved" in feedback:
#             print("The letter has been approved!")
#             return current_letter
        
#         else:
#             print("reed modification...................")
#             # If not approved, refine the letter based on feedback
#             refinement_prompt = f"""
#             The evaluator provided the following feedback on your previous letter:
#             {feedback}

#             Based on this feedback, refine the cover letter to address the issues and improve its quality. 
#             Here are the original details for context:

#             **Candidate's CV**:
#             {cv_data}

#             **Job Description**:
#             {job_description}

#             **Previous cover Letter**:
#             {current_letter}

#             Refine the letter while maintaining a professional tone and aligning it with the feedback.
#             """
#             current_letter = llm4generator_extractor(refinement_prompt)

#     print("Max iterations reached. Returning the latest version of the letter.")
#     return current_letter

In [ ]:
cover_letter=feedback_loop(cv_data, job_description, max_iterations=2)
# with open("cover_letter4all.txt", "w") as file:
#     file.write(cover_letter)

In [ ]:
with open("cover_letter.txt", "w") as file:
    file.write(cover_letter)

In [ ]:
def user_feedback_fun(user_feedback,cv_data,job_description,current_letter):
        refinement_prompt = f"""
        The evaluator provided the following feedback on your previous cover letter:
        {user_feedback}

        Based on this feedback, refine the cover letter to improve its quality while addressing the issues highlighted.
        Ensure it remains professional and well-aligned with the job description.

        Here are the details for context:

        **Candidate's CV**:
        {cv_data}

        **Job Description**:
        {job_description}

        **Current Cover Letter**:
        {current_letter}

        Generate a new and improved version of the cover letter.
        
        you must only add the new cover letter dont add the feedback or job Description
        the cover letter must End with the candidate’s name as extracted for CV.

        """
        
        # Generate the refined cover letter
        refined_COV = llm4generator_extractor(refinement_prompt)
        print("refined_COV")
        print(refined_COV)
        print("*****************************************************")
        return refined_COV

In [ ]:
user_feedback="""
please add Bassant Elsayed after Sincerely,

"""
final_COV=user_feedback_fun(user_feedback,cv_data,job_description,cover_letter)
with open("final_cv.txt", "w") as file:
    file.write(final_COV)

In [ ]:
!pip install spacy nltk rake-nltk
!python -m spacy download en_core_web_sm

In [8]:
!{sys.executable} -m pip install spacy nltk rake-nltk
!{sys.executable} -m python -m spacy download en_core_web_sm

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached rake_nltk-1.0.6-py3-none-any.whl.metadata (6.4 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached rake_nltk-1.0.6-py3-none-any.whl (9.1 kB)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
d:\olga_bassant_roadmap\B2B_project\B2B\myenv\Scripts\python.exe: No module named python


In [9]:
with open("cv.txt", "r") as file:
    cv_text = file.read()

In [10]:
from nltk.corpus import stopwords
from rake_nltk import Rake
import spacy
import nltk
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
from rake_nltk import Rake
import spacy
# import nltk

# # Download NLTK stopwords (required for RAKE)
# nltk.download('stopwords')

# Initialize RAKE and spaCy
rake = Rake()
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')

# Input 1: CV
cv_text = """
Name: John Doe
Email: johndoe@example.com
Phone: +1234567890

Education:
- Bachelor of Science in Computer Science, XYZ University, 2015-2019
- Master of Science in Data Science, ABC University, 2019-2021

Experience:
- Software Engineer at TechCorp, 2021-Present
- Data Analyst at DataWorks, 2019-2021

Skills:
- Python
- Machine Learning
- SQL
- Data Analysis
"""

# Input 2: Job Description
job_description_text = """
Machine Learning Engineer, Network Assurance Data Platform
Rolle, Switzerland
Please note that we have a hybrid approach to work and are looking for someone who can come into our offices in Rolle two days per week.
Who We Are
Cisco ThousandEyes is a Digital Experience Assurance platform that empowers organizations to deliver flawless digital experiences across every network – even the ones they don’t own. Powered by AI and an unmatched set of cloud, internet and enterprise network telemetry data, ThousandEyes enables IT teams to proactively detect, diagnose, and remediate issues – before they impact end- user experiences.

ThousandEyes is deeply integrated across the entire Cisco technology portfolio and beyond, helping customers deploy at scale while also delivering AI-powered assurance insights within Cisco’s leading Networking, Security, Collaboration, and Observability portfolios.

About The Role
As an AI Researcher on the Network Assurance Data Platform team, you will be responsible for building the next-generation routing engine for the Internet using Data Science, Machine Learning (ML), and Artificial Intelligence (AI). The size, diversity, and richness of our data are simply unmatched. The Internet is changing fast, and we are too. The objective of this confidential product is to adopt a highly novel and ambitious approach for optimizing networks to better serve applications.

What You’ll Do
You and your team will be designing and implementing large-scale machine learning pipelines. Our technology stack includes Python as well as a wide range of internal tools built on top of Apache Spark, TensorFlow, and other Big Data technologies.
"""

# Function to extract key phrases
def extract_key_phrases(text):
    # Extract using RAKE
    rake.extract_keywords_from_text(text)
    rake_phrases = rake.get_ranked_phrases()
    
    # Extract using spaCy
    doc = nlp(text)
    spacy_phrases = set()
    for chunk in doc.noun_chunks:
        spacy_phrases.add(chunk.text)
    for ent in doc.ents:
        spacy_phrases.add(ent.text)
    
    # Combine and deduplicate
    return list(set(rake_phrases + list(spacy_phrases)))

# Extract key phrases from CV and job description
cv_phrases = extract_key_phrases(cv_text)
job_phrases = extract_key_phrases(job_description_text)

# Merge results and remove duplicates
key_phrases = list(set(cv_phrases + job_phrases))

# Filter out irrelevant phrases (optional)
# irrelevant_phrases = ["name", "email", "phone", "rolle", "switzerland"]
key_phrases = [phrase for phrase in key_phrases]
# key_phrases = [phrase for phrase in key_phrases if phrase.lower() not in irrelevant_phrases]

print("Key Phrases:", key_phrases)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LOQ\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\LOQ/nltk_data'
    - 'd:\\olga_bassant_roadmap\\B2B_project\\B2B\\myenv\\nltk_data'
    - 'd:\\olga_bassant_roadmap\\B2B_project\\B2B\\myenv\\share\\nltk_data'
    - 'd:\\olga_bassant_roadmap\\B2B_project\\B2B\\myenv\\lib\\nltk_data'
    - 'C:\\Users\\LOQ\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import recall_score

def extract_keywords(text):
    vectorizer = CountVectorizer(stop_words="english")
    X = vectorizer.fit_transform([text])
    print(X)
    return set(vectorizer.get_feature_names_out())

def compute_recall(cv, job_desc, cover_letter):
    # Extract unique keywords
    cv_keywords = extract_keywords(cv)
    jd_keywords = extract_keywords(job_desc)
    cl_keywords = extract_keywords(cover_letter)

    # Combine CV and Job Description keywords
    all_keywords = cv_keywords.union(jd_keywords)

    # Compute recall
    recall = len(cl_keywords.intersection(all_keywords)) / len(all_keywords) if len(all_keywords) > 0 else 0
    return recall

# Example inputs
cv_text = "Machine Learning Engineer with experience in Python, TensorFlow, and NLP."
job_desc_text = "Looking for an ML Engineer skilled in Python, NLP, and deploying AI models."
cover_letter_text = "I am an experienced Machine Learning Engineer with expertise in Python and NLP."

# Compute context recall
recall_score = compute_recall(cv_text, job_desc_text, cover_letter_text)
print(f"Context Recall: {recall_score:.2f}")


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 7 stored elements and shape (1, 7)>
  Coords	Values
  (0, 3)	1
  (0, 2)	1
  (0, 0)	1
  (0, 1)	1
  (0, 5)	1
  (0, 6)	1
  (0, 4)	1
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 9 stored elements and shape (1, 9)>
  Coords	Values
  (0, 3)	1
  (0, 4)	1
  (0, 2)	1
  (0, 8)	1
  (0, 7)	1
  (0, 6)	1
  (0, 1)	1
  (0, 0)	1
  (0, 5)	1
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 7 stored elements and shape (1, 7)>
  Coords	Values
  (0, 1)	1
  (0, 4)	1
  (0, 3)	1
  (0, 0)	1
  (0, 2)	1
  (0, 6)	1
  (0, 5)	1
Context Recall: 0.38


In [24]:
with open("cv.txt", "r") as file:
    cv_text = file.read()
with open("job Description1.txt", "r") as file:
    job_desc_text = file.read() 
with open("cover_letter.txt", "r") as file:
    cv_text = file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'cover_letter.txt'

In [ ]:
D:\olga_bassant_roadmap\B2B_project\B2B\app\llm4allV1.1.ipynb

In [20]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def compute_bert_recall(cv, job_desc, cover_letter):
    # Load pre-trained BERT model for embeddings
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Get sentence embeddings
    cv_embedding = model.encode(cv)
    jd_embedding = model.encode(job_desc)
    cl_embedding = model.encode(cover_letter)

    # Compute combined embedding for CV + JD
    combined_cv_jd = np.mean([cv_embedding, jd_embedding], axis=0)

    # Compute cosine similarity between Cover Letter and (CV + JD)
    similarity = cosine_similarity([combined_cv_jd], [cl_embedding])[0][0]
    
    return similarity

# Compute BERT-based recall
# Example inputs
# cv_text = "Machine Learning Engineer with experience in Python, TensorFlow, and NLP."
# job_desc_text = "Looking for an ML Engineer skilled in Python, NLP, and deploying AI models."
# cover_letter_text = "I am an experienced Machine Learning Engineer with expertise in Python and NLP."


bert_recall = compute_bert_recall(cv_text, job_desc_text, cover_letter_text)
print(f"BERT Context Recall: {bert_recall:.2f}")


BERT Context Recall: 0.49


In [27]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained BERT model
model = SentenceTransformer("all-MiniLM-L6-v2")  # Lightweight and effective

# Function to read text from a file
def read_text_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read().strip()

# Load CV, Job Description, and Cover Letter from files
cv_text = read_text_file("cv.txt")
job_desc_text = read_text_file("job Description1.txt")
cover_letter_text = read_text_file("cover_letter.txt")

# Split cover letter into sentences
cover_letter_sentences = cover_letter_text.split(". ")  # Splitting by sentence

# Combine CV and Job Description for comparison
cv_jd_texts = [cv_text, job_desc_text]
cv_jd_embeddings = model.encode(cv_jd_texts, convert_to_tensor=True)

# Encode cover letter sentences
cover_letter_embeddings = model.encode(cover_letter_sentences, convert_to_tensor=True)

# Identify weak sentences
weak_sentences = []

for i, sentence in enumerate(cover_letter_sentences):
    similarity_scores = util.pytorch_cos_sim(cover_letter_embeddings[i], cv_jd_embeddings)
    max_similarity = similarity_scores.max().item()  # Get highest similarity score

    # If similarity is below a threshold (e.g., 0.6), consider it weak
    if max_similarity < 0.6:
        weak_sentences.append((sentence, max_similarity))

# Print weak sentences
if weak_sentences:
    print("Weak Sentences in Cover Letter (low relevance to CV/Job Description):\n")
    for sent, score in weak_sentences:
        print(f" {sent} (Similarity: {score:.2f})")
        print("************************************************************************************")
else:
    print(" No weak sentences detected! Your cover letter is well-aligned.")


Weak Sentences in Cover Letter (low relevance to CV/Job Description):

 As a seasoned Machine Learning Engineer with a background in Electronics and Electrical Communication Engineering, I bring a unique blend of technical expertise and problem-solving abilities to the table (Similarity: 0.57)
************************************************************************************
 Additionally, I designed intelligent systems and tools for RF planning and optimization, enhancing decision-making processes (Similarity: 0.54)
************************************************************************************
 These experiences have honed my ability to develop innovative solutions that drive business success.

My education in Electronics and Electrical Communication Engineering has provided me with a solid foundation in programming & frameworks (Python, PyTorch, TensorFlow, Hugging Face Transformers), NLP & LLM expertise (fine-tuning Transformer-based models, prompt engineering, text classifi